In [24]:
import pickle
import json
import geopandas 
import pandas as pd
pd.set_option('max_colwidth', 200)
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as pltc
from shapely.geometry import Point, Polygon, LineString
import seaborn as sns 
sns.set()
import os
from os import listdir
from os.path import isfile, join
import descartes
import csv
import numpy as np
%matplotlib inline
from random import sample
from math import sin, cos, sqrt, atan2, radians, acos
import numpy as np
import math
import copy
import time

lakes = geopandas.read_file("Lakes_and_Rivers")
madison = geopandas.read_file("City_Limit")


In [2]:
def save_obj(obj, name ):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [3]:
g=load_obj('graph_final_fixed')


In [7]:
# with open('obj/'+'data.json', 'w') as fp:
#     json.dump(g, fp)

In [ ]:
with open('data.json', 'r') as fp:
    data = json.load(fp)

In [12]:

def get_stop(node):
    return int(node.split("_")[0])
    
def get_seconds(node):
    return int(node.split("_")[1])
    
# depth-first search through bus system
#
# graph: dict, key=node name, val=edge list
# node: node name
# max_time: last time (in seconds) beyond which we don't search (None if no limit)
# visited: set of nodes visited already (to avoid repeats)
def dfs(graph, node, max_time=None, visited=None, parent_node= None):
    if visited == None:
        visited = set()
    node_time = get_seconds(node)
    if max_time != None and node_time > max_time:
        return visited

    if node in visited:
        return visited
    
    if parent_node:
        trip_id = parent_node['trip_id'] 
    else: 
        trip_id = None
        
    visited.add(node)
    for edge in graph[node]:
            if not trip_id:
                dfs(graph, edge["destination"], max_time, visited, edge)
            else:
                if 'DROP OFF' in edge['full_bus_name']:
                    if edge['trip_id'] == trip_id:
                        dfs(graph, edge["destination"], max_time, visited, edge)

                    else: 
                        continue 
                else:
                    dfs(graph, edge["destination"], max_time, visited, edge)
    return visited

def stops_reachable_percent(start, hours=None):
    limit = None
    if hours != None:
        limit = get_seconds(start) + hours * 3600 # hours to seconds
    reachable = dfs(g, start, limit)
    reachable_stops = {get_stop(node) for node in reachable}
    all_stops = {get_stop(node) for node in g.keys()}
    hour_cover = (len(reachable_stops) / len(all_stops))
    return hour_cover, reachable   

# # where can we get if we start at this place/time, and have no time limit?
# print(stops_reachable_percent('1561_25223'))

# # one hour limit?
# print(stops_reachable_percent('1379_25260', 1))

# # two hour limit?
# print(stops_reachable_percent('1561_25223', 2))


In [13]:
hour_coverage = {}
dfs_output = {}
past_percent = -1
for i,node in enumerate(g):
    hour_coverage[node], dfs_output[node] = stops_reachable_percent(node, 1)
    if i*100//len(g) != past_percent:
        past_percent = i*100//len(g)
        if str(0) in str(past_percent):
            print("%d%% done" % past_percent)



0% done
1% done
2% done
3% done
4% done
5% done
6% done
7% done
8% done
9% done
10% done
11% done
12% done
13% done
14% done
15% done
16% done
17% done
18% done
19% done
20% done
21% done
22% done
23% done
24% done
25% done
26% done
27% done
28% done
29% done
30% done
31% done
32% done
33% done
34% done
35% done
36% done
37% done
38% done
39% done
40% done
41% done
42% done
43% done
44% done
45% done
46% done
47% done
48% done
49% done
50% done
51% done
52% done
53% done
54% done
55% done
56% done
57% done
58% done
59% done
60% done
61% done
62% done
63% done
64% done
65% done
66% done
67% done
68% done
69% done
70% done
71% done
72% done
73% done
74% done
75% done
76% done
77% done
78% done
79% done
80% done
81% done
82% done
83% done
84% done
85% done
86% done
87% done
88% done
89% done
90% done
91% done
92% done
93% done
94% done
95% done
96% done
97% done
98% done
99% done


In [22]:
best = max(hour_coverage.values())
print([n for n in g if hour_coverage[n] == best])

['1787_26047']


In [17]:
dfs_output

{'3733_27986',
 '1927_29210',
 '9339_26986',
 '7167_29296',
 '160_29549',
 '2125_29597',
 '1798_29277',
 '1513_28012',
 '1842_28104',
 '2048_28760',
 '1787_28127',
 '6134_27835',
 '839_29113',
 '2978_26999',
 '4354_27818',
 '1280_29273',
 '1311_27834',
 '862_29547',
 '1710_28767',
 '168_27410',
 '2702_27553',
 '532_27883',
 '388_28085',
 '8138_29005',
 '1809_29219',
 '1310_27537',
 '898_26405',
 '2395_29595',
 '1131_26845',
 '2088_29123',
 '6511_28610',
 '2883_28650',
 '1309_29009',
 '2125_27917',
 '124_28081',
 '2500_29479',
 '2712_28548',
 '2901_28506',
 '1323_27497',
 '4836_28500',
 '845_29093',
 '2125_28515',
 '6965_28974',
 '2940_27999',
 '2647_28345',
 '9750_29035',
 '456_29210',
 '568_29034',
 '1759_29336',
 '1855_28949',
 '2131_29440',
 '2131_28860',
 '898_29015',
 '845_29390',
 '1250_29566',
 '1927_27671',
 '2393_27144',
 '252_28418',
 '2201_28144',
 '2195_29580',
 '8816_28746',
 '2125_28862',
 '1397_26451',
 '1839_28053',
 '1996_28867',
 '3105_27234',
 '2376_29160',
 '2946_28

In [18]:
save_obj(dfs_output, 'dfs_output_final')

In [19]:
save_obj(hour_coverage, 'hour_coverage_final')